In [1]:
## Install Coqui TTS
! pip install -U pip
! pip install coqui-tts==0.26.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 51.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 90.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 85.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 125.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 63.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 116.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 861

# Imports

In [2]:
import os

import torch
from trainer import Trainer, TrainerArgs

from TTS.bin.compute_embeddings import compute_embeddings
from TTS.config import load_config
from TTS.config.shared_configs import BaseDatasetConfig
from TTS.tts.configs.vits_config import VitsConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.models.vits import CharactersConfig, Vits, VitsArgs, VitsAudioConfig
from TTS.tts.utils.speakers import SpeakerManager
from TTS.tts.utils.managers import save_file
from tqdm import tqdm
import json
import gdown
import tarfile

torch.set_num_threads(24)

2025-05-17 20:28:20.458317: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747513700.646878      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747513700.699290      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
OUT_PATH = "/kaggle/working/yourtts_tamazight"
LANG_NAME = "tamazight"
ISO = "zgh"

# Name of the run for the Trainer
RUN_NAME = f"YourTTS-{LANG_NAME.capitalize()}"

# Create output directory if it doesn't exist
if not os.path.exists(OUT_PATH):
    os.makedirs(OUT_PATH)

In [4]:
# This paramter is useful to debug, it skips the training epochs and just do the evaluation  and produce the test sentences
SKIP_TRAIN_EPOCH = False

# Set here the batch size to be used in training and evaluation
BATCH_SIZE = 40

# Note: If you add new datasets, please make sure that the dataset sampling rate and this parameter are matching, otherwise resample your audios
SAMPLE_RATE = 22050

# Max audio length in seconds to be used in training
MAX_AUDIO_LEN_IN_SECONDS = 15
# Min audio length in seconds to be used in training
MIN_AUDIO_LEN_IN_SECONDS = 0.8

In [5]:
# If you want to do transfer learning and speedup your training you can set here the path to the CML-TTS available checkpoint that can be downloaded here:  https://drive.google.com/u/2/uc?id=1yDCSJ1pFZQTHhL09GMbOrdjcPULApa0p
RESTORE_PATH = os.path.join(OUT_PATH, "checkpoints_yourtts_cml_tts_dataset/best_model.pth")

URL = "https://drive.google.com/u/2/uc?id=1yDCSJ1pFZQTHhL09GMbOrdjcPULApa0p"
OUTPUT_CHECKPOINTS_FILEPATH = os.path.join(OUT_PATH, "checkpoints_yourtts_cml_tts_dataset.tar.bz")

# Download the CML-TTS checkpoint if it does not exist
if not os.path.exists(RESTORE_PATH):
    print(f"Downloading the CML-TTS checkpoint from {URL}")
    gdown.download(url=URL, output=OUTPUT_CHECKPOINTS_FILEPATH, quiet=False, fuzzy=True)
    with tarfile.open(OUTPUT_CHECKPOINTS_FILEPATH, "r:bz2") as tar:
        tar.extractall(OUT_PATH)
else:
    print(f"Checkpoint already exists at {RESTORE_PATH}")

Downloading...
From (original): https://drive.google.com/uc?id=1yDCSJ1pFZQTHhL09GMbOrdjcPULApa0p
From (redirected): https://drive.google.com/uc?id=1yDCSJ1pFZQTHhL09GMbOrdjcPULApa0p&confirm=t&uuid=e54dca8b-21af-428c-8e58-50dbd392cbb3
To: /kaggle/working/yourtts_tamazight/checkpoints_yourtts_cml_tts_dataset.tar.bz
100%|██████████| 1.04G/1.04G [00:44<00:00, 23.4MB/s]


# Prepare dataset

In [6]:
from datasets import load_dataset, Audio

ds = load_dataset("HackAI-2025/tts_dataset_30h", split="train")

README.md:   0%|          | 0.00/363 [00:00<?, ?B/s]

train-00000-of-00010.parquet:   0%|          | 0.00/444M [00:00<?, ?B/s]

train-00001-of-00010.parquet:   0%|          | 0.00/444M [00:00<?, ?B/s]

train-00002-of-00010.parquet:   0%|          | 0.00/442M [00:00<?, ?B/s]

train-00003-of-00010.parquet:   0%|          | 0.00/444M [00:00<?, ?B/s]

...

train-00006-of-00010.parquet:   0%|          | 0.00/452M [00:00<?, ?B/s]

train-00007-of-00010.parquet:   0%|          | 0.00/437M [00:00<?, ?B/s]

train-00008-of-00010.parquet:   0%|          | 0.00/448M [00:00<?, ?B/s]

train-00009-of-00010.parquet:   0%|          | 0.00/456M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/19568 [00:00<?, ? examples/s]

In [7]:
ds[0]

{'audio': {'path': '8000_04_NUM_shi_29_0003.wav',
  'array': array([ 0.        ,  0.        ,  0.        , ..., -0.04641724,
         -0.04385376, -0.0406189 ]),
  'sampling_rate': 22050},
 'text': 'ⴷ ⵢⴰⵏ ⵓⵏⴽⴽⵓⵔ ⴰ ⵉⴳ ⴰⵙⴰⴽⵎⵓⵔ ⵏ ⵍⵎⵄⵚⵉⵢⵜ ⴰ ⴼⵍⵍⴰⵡⵏ ⵉⴽⴽⵙ ⵍⴽⴼⴼⴰⵔⵜ.',
 'duration': 4.036734693877551}

In [8]:
# Listen to a random sample
import IPython
import random

random_index = random.randint(0, len(ds))
sample = ds[random_index]
print("Transcription: ", sample['text'])
print("Duration: ", sample['duration'])
IPython.display.Audio(sample['audio']['array'], rate=sample['audio']['sampling_rate'])

Transcription:  ⵎⴰ ⵙ ⵔⴰ ⵏⵣⵣⵉⵖⵥⴰⵏ ⵉⵅⴼ ⵏⵏⵅ?
Duration:  2.238185941043084


In [9]:
# Get statistics about the duration of audio files
durations = ds['duration']
print("Average duration: ", sum(durations) / len(durations))
print("Max duration: ", max(durations))
print("Min duration: ", min(durations))
print(f"Total duration in hours: {sum(durations) / 3600:.2}")

Average duration:  5.633002175795363
Max duration:  9.991836734693878
Min duration:  1.0391383219954649
Total duration in hours: 3.1e+01


In [10]:
# Get character count frequencies
from collections import Counter

def get_char_counts(ds):
    all_text = ds['text']
    char_counts = Counter(''.join(all_text))
    # Sort
    char_counts = dict(sorted(char_counts.items(), key=lambda item: item[1], reverse=True))
    return char_counts

char_counts = get_char_counts(ds)
char_counts

{' ': 269332,
 'ⴰ': 152739,
 'ⵏ': 127420,
 'ⵉ': 121016,
 'ⵍ': 68839,
 'ⵜ': 67032,
 'ⵙ': 62299,
 'ⵓ': 55868,
 'ⴷ': 51816,
 'ⵔ': 50174,
 'ⵎ': 42462,
 'ⵢ': 30832,
 'ⴽ': 30659,
 'ⴳ': 23586,
 'ⵖ': 23403,
 'ⴱ': 23072,
 'ⵡ': 22764,
 'ⴼ': 16913,
 '.': 16841,
 ',': 12076,
 'ⵣ': 10999,
 'ⵅ': 9741,
 'ⵕ': 9295,
 'ⵛ': 8469,
 'ⵄ': 7597,
 'ⴹ': 6378,
 'ⵃ': 6370,
 'ⵇ': 5584,
 'ⵊ': 4991,
 'ⵀ': 4361,
 ':': 3961,
 'ⵥ': 3759,
 'ⵚ': 3732,
 '?': 1984,
 'ⵟ': 1526,
 'ⵯ': 1220,
 '!': 749,
 'ⴻ': 48}

In [11]:
PUNCT = set(' !,.:?')

CHARS = char_counts.keys()
CHARS = set(CHARS)

# Remove punctuation from character set
CHARS = CHARS - PUNCT
CHARS = sorted(list(CHARS))
PUNCT = sorted(list(PUNCT))

print("Punctuation: ", PUNCT)
print("Character set: ", CHARS)

Punctuation:  [' ', '!', ',', '.', ':', '?']
Character set:  ['ⴰ', 'ⴱ', 'ⴳ', 'ⴷ', 'ⴹ', 'ⴻ', 'ⴼ', 'ⴽ', 'ⵀ', 'ⵃ', 'ⵄ', 'ⵅ', 'ⵇ', 'ⵉ', 'ⵊ', 'ⵍ', 'ⵎ', 'ⵏ', 'ⵓ', 'ⵔ', 'ⵕ', 'ⵖ', 'ⵙ', 'ⵚ', 'ⵛ', 'ⵜ', 'ⵟ', 'ⵡ', 'ⵢ', 'ⵣ', 'ⵥ', 'ⵯ']


In [12]:
# Create character config
from TTS.tts.configs.shared_configs import CharactersConfig

characters = CharactersConfig(
    characters_class="TTS.tts.models.vits.VitsCharacters",
    pad="_",
    eos="&",
    bos="*",
    blank=None,
    characters="".join(CHARS),
    punctuations="".join(PUNCT),
)

In [13]:
import pandas as pd
import librosa
import soundfile as sf
import os
from tqdm import tqdm

# Create the output directory if it doesn't exist
output_dir = "/kaggle/temp/zgh_tts_dataset"
wavs_dir = os.path.join(output_dir, "wavs")
os.makedirs(wavs_dir, exist_ok=True)

# Create the metadata file
metadata_file = os.path.join(output_dir, "metadata.csv")
metadata = []

# Iterate over the dataset
for i, sample in tqdm(enumerate(ds), total=len(ds)):
    # Get the audio data and text
    audio_array = sample['audio']['array']
    text = sample['text']

    # Create a unique file name
    file_name_no_ext = f"{i:05d}"
    file_name = f"{file_name_no_ext}.wav"
    file_path = os.path.join(wavs_dir, file_name)

    # Save the audio file at 22050 Hz
    sf.write(file_path, audio_array, 22050)

    # Append to the metadata
    metadata.append(f"wavs/{file_name}||{text}|yan|")

# Write the metadata to the csv file
with open(metadata_file, 'w') as f:
  for line in metadata:
    f.write(line + '\n')


100%|██████████| 19568/19568 [03:02<00:00, 107.46it/s]


In [14]:
!ls -lh $output_dir

total 4.3M
-rw-r--r-- 1 root root 3.8M May 17 20:37 metadata.csv
drwxr-xr-x 2 root root 548K May 17 20:37 wavs


In [15]:
!head $output_dir/metadata.csv

wavs/00000.wav||ⴷ ⵢⴰⵏ ⵓⵏⴽⴽⵓⵔ ⴰ ⵉⴳ ⴰⵙⴰⴽⵎⵓⵔ ⵏ ⵍⵎⵄⵚⵉⵢⵜ ⴰ ⴼⵍⵍⴰⵡⵏ ⵉⴽⴽⵙ ⵍⴽⴼⴼⴰⵔⵜ.|yan|
wavs/00001.wav||ⵉⵎⵍ ⵉⵍⵎⵎⴰ ⵎⵓⵙⴰ ⵉ ⴰⵢⵜ ⵉⵙⵔⴰⵉⵍ ⴰⵢⵍⵍⵉ ⴼ ⵜ ⵉⴷ ⴽⵓⵍⵍⵓ ⵢⵓⵚⵚⴰ ⵙⵉⴷⵉ ⵕⴱⴱⵉ.|yan|
wavs/00002.wav||ⵍⵍⵉⵖ ⵉⵙⵙⴼⵍⴷ ⴷⴰⵡⵓⴷ ⴼ ⵎⴰⵢⴰⵏⵏ ⵢⴰⵣⵏ ⵢⵓⴰⴱ ⴷ ⵉⵎⴷⵡⴰⵙ ⴽⵓⵍⵍⵓ ⵏ ⵍⵄⵙⴽⵔ ⵏⵏⵙ.|yan|
wavs/00003.wav||ⴼⴼⵖⵏ ⴷ ⴰⵢⵜ ⵄⴰⵎⵎⵓⵏ ⵎⴷⴰⵔⴰⵙⵏ ⵖ ⵜⴰⵎⴰ ⵏ ⵉⵎⵉ ⵏ ⵜⵎⴷⵉⵏⵜ ⴼ ⴰ ⴷⵉⴷⵙⵏ ⵎⵎⴰⵖⵏ, ⵉⴳⵍⴷⴰⵏ ⵍⵍⵉ ⴷ ⵓⵛⴽⴰⵏⵉⵏ ⵏⵜⵜⵏⵉ ⵍⵍⴰⵏ ⵉⵏⵏ ⵖ ⵉⴳⵔⴰⵏ ⵖ ⴱⵕⵕⴰ.|yan|
wavs/00004.wav||ⵍⵍⵉⵖ ⵉⵙⵙⵏ ⵢⵓⴰⴱ ⵉⵙ ⴰⵙ ⴷ ⵉⴽⴽⴰ ⵍⵄⴷⵓ ⵍⴳⴷⴷⴰⵎ ⴷ ⵜⵖⵕⴹⵉⵏ, ⵉⵙⵜⵉ ⵉⵄⵙⴽⵔⵉⵢⵏ ⵍⵍⵉ ⴱⴰⵀⵔⴰ ⴷⵓⵙⵏⵉⵏ ⴰⴷ ⵎⵎⴰⵖⵏ ⴷ ⴰⵢⵜ ⴰⵔⴰⵎ.|yan|
wavs/00005.wav||ⵉⴳ ⴳⵎⴰⵙ ⴰⴱⵉⵛⴰⵢ ⴼ ⵍⵄⵙⴽⵔ ⴰⵏⵏ ⵢⴰⴹⵏⵉⵏ, ⵎⴷⴰⵔⴰⵙⵏ ⴰⴷ ⵎⵎⴰⵖⵏ ⴷ ⴰⵢⵜ ⵄⴰⵎⵎⵓⵏ.|yan|
wavs/00006.wav||ⵉⵏⵏⴰ ⵢⵙ ⵢⵓⴰⴱ ⵉ ⴰⴱⵉⵛⴰⵢ: ⵉⵖ ⵉⵢⵉ ⵏⵏⵔⴰⵏ ⴰⵢⵜ ⴰⵔⴰⵎ ⵜⵃⴰⵔⵔⵓⵜ ⴷ ⴰⴷ ⵉⵢⵉ ⵜⴼⵓⴽⴽⵓⵜ, ⵉⵖ ⴽ ⵏⵏⵔⴰⵏ ⴰⵢⵜ ⵄⴰⵎⵎⵓⵏ ⵃⴰⵔⵔⵓⵅ ⵏⵏ ⴰ ⴽ ⵉⴷ ⴼⵓⴽⴽⵓⵅ.|yan|
wavs/00007.wav||ⴳ ⴰⵎⴷⵡⴰⵙ ⵜⵣⵄⵎⵜ ⴰ ⵏⵎⵎⴰⵖ ⴼ ⴰⵢⵜⴷⴰⵔⵏⵅ ⴷ ⵜⵎⴷⵉⵏⵉⵏ ⵏ ⵕⴱⴱⵉ ⵍⵍⵉ ⵏⵜⵜⵄⴱⴰⴷ, ⵀⴰⵏⵏ ⵙⵉⴷⵉ ⵕⴱⴱⵉ ⵔⴰ ⵉⵙⴽⵔ ⴰⵢⵍⵍⵉ ⴷⴰⵔⵙ ⵉⴼⵓⵍⴽⵉⵏ.|yan|
wavs/00008.wav||ⵍⵍⵉⵖ ⵥⵕⴰⵏ ⴰⵢⵜ ⵄⴰⵎⵎⵓⵏ ⴰⵢⵜ ⴰⵔⴰⵎ ⵔⵡⵍⵏ, ⵔⵡⵍⵏ ⵓⵍⴰ ⵏⵜⵜⵏⵉ ⵖ ⵓⵎⵏⵉⴷ ⵏ ⴰⴱⵉⵛⴰⵢ ⴽⵛⵎⵏ ⵙ ⵜⵎⴷⵉⵏⵜ.|yan|
wavs/00009.wav||ⵢⴰⴹⵓ ⴷ ⵉⵍⵎⵎⴰ ⵢ

In [16]:
dataset_conf = BaseDatasetConfig(
    formatter="brspeech", meta_file_train="metadata.csv", path=output_dir, dataset_name="zgh_tts_dataset", language=ISO
)

In [17]:
### Extract speaker embeddings
SPEAKER_ENCODER_CHECKPOINT_PATH = (
    "https://github.com/coqui-ai/TTS/releases/download/speaker_encoder_model/model_se.pth.tar"
)
SPEAKER_ENCODER_CONFIG_PATH = "https://github.com/coqui-ai/TTS/releases/download/speaker_encoder_model/config_se.json"

D_VECTOR_FILES = []  # List of speaker embeddings/d-vectors to be used during the training

# Checks if the speakers embeddings are already computated, if not compute it
embeddings_file = os.path.join(dataset_conf.path, "speakers.pth")
if not os.path.isfile(embeddings_file):
    print(f">>> Computing the speaker embeddings for the {dataset_conf.dataset_name} dataset")
    compute_embeddings(
        SPEAKER_ENCODER_CHECKPOINT_PATH,
        SPEAKER_ENCODER_CONFIG_PATH,
        embeddings_file,
        formatter_name=dataset_conf.formatter,
        dataset_name=dataset_conf.dataset_name,
        dataset_path=dataset_conf.path,
        meta_file_train=dataset_conf.meta_file_train,
        meta_file_val=dataset_conf.meta_file_val,
    )
D_VECTOR_FILES.append(embeddings_file)

>>> Computing the speaker embeddings for the zgh_tts_dataset dataset


100%|██████████| 19568/19568 [06:01<00:00, 54.17it/s]


Speaker embeddings saved at: /kaggle/temp/zgh_tts_dataset/speakers.pth


In [18]:
# Audio config used in training.
audio_config = VitsAudioConfig(
    sample_rate=SAMPLE_RATE,
    hop_length=256,
    win_length=1024,
    fft_size=1024,
    mel_fmin=0.0,
    mel_fmax=None,
    num_mels=80,
)

In [19]:
# Init VITSArgs setting the arguments that are needed for the YourTTS model
model_args = VitsArgs(
    spec_segment_size=62,
    hidden_channels=192,
    hidden_channels_ffn_text_encoder=768,
    num_heads_text_encoder=2,
    num_layers_text_encoder=10,
    kernel_size_text_encoder=3,
    dropout_p_text_encoder=0.1,
    d_vector_file=D_VECTOR_FILES,
    use_d_vector_file=True,
    d_vector_dim=512,
    speaker_encoder_model_path=SPEAKER_ENCODER_CHECKPOINT_PATH,
    speaker_encoder_config_path=SPEAKER_ENCODER_CONFIG_PATH,
    resblock_type_decoder="2",  # In the paper, we accidentally trained the YourTTS using ResNet blocks type 2, if you like you can use the ResNet blocks type 1 like the VITS model
    # Useful parameters to enable the Speaker Consistency Loss (SCL) described in the paper
    use_speaker_encoder_as_loss=False,
    # Useful parameters to enable multilingual training
    use_language_embedding=True,
    embedded_language_dim=4,
)

In [20]:
TEST_SENTENCES = [
    ["ⴰⵣⵓⵍ ⴰⵢⵜⵎⴰ ⴷ ⵉⵙⵙⵜⵎⴰ!", "yan", None, "zgh"],
]

In [21]:
# General training config, here you can change the batch size and others useful parameters
config = VitsConfig(
    epochs=25,
    output_path=OUT_PATH,
    model_args=model_args,
    run_name=RUN_NAME,
    project_name="YourTTS",
    run_description=f"""
            - YourTTS trained using the {LANG_NAME.capitalize()} dataset.
        """,
    dashboard_logger="tensorboard",
    logger_uri=None,
    audio=audio_config,
    batch_size=BATCH_SIZE,
    batch_group_size=4,
    eval_batch_size=BATCH_SIZE,
    num_loader_workers=8,
    # eval_split_max_size=256,
    print_step=25,
    plot_step=50,
    # log_model_step=1000,
    save_step=1000,
    save_n_checkpoints=5,
    save_checkpoints=True,
    target_loss="loss_1",
    print_eval=True,
    compute_input_seq_cache=True,
    add_blank=True,
    text_cleaner="no_cleaners",
    characters=characters,
    phoneme_cache_path=None,
    precompute_num_workers=12,
    start_by_longest=True,
    datasets=[dataset_conf],
    cudnn_benchmark=False,
    min_audio_len=int(SAMPLE_RATE * MIN_AUDIO_LEN_IN_SECONDS),
    max_audio_len=SAMPLE_RATE * MAX_AUDIO_LEN_IN_SECONDS,
    mixed_precision=True,
    test_sentences=TEST_SENTENCES,
    # Enable the weighted sampler
    # use_weighted_sampler=True,
    # Ensures that all speakers are seen in the training batch equally no matter how many samples each speaker has
    # weighted_sampler_attrs={"language": 1.0, "speaker_name": 1.0},
    # weighted_sampler_attrs={"language": 1.0},
    # weighted_sampler_multipliers={
    #     # "speaker_name": {
    #     # you can force the batching scheme to give a higher weight to a certain speaker and then this speaker will appears more frequently on the batch.
    #     # It will speedup the speaker adaptation process. Considering the CML train dataset and "new_speaker" as the speaker name of the speaker that you want to adapt.
    #     # The line above will make the balancer consider the "new_speaker" as 106 speakers so 1/4 of the number of speakers present on CML dataset.
    #     # 'new_speaker': 106, # (CML tot. train speaker)/4 = (424/4) = 106
    #     # }
    # },
    # It defines the Speaker Consistency Loss (SCL) α to 9 like the YourTTS paper
    speaker_encoder_loss_alpha=9.0,
)

In [22]:
# Load all the datasets samples and split traning and evaluation sets
train_samples, eval_samples = load_tts_samples(
    config.datasets,
    eval_split=True,
    eval_split_max_size=config.eval_split_max_size,
    eval_split_size=config.eval_split_size,
)
print(f"Loaded {len(train_samples)} train samples")
print(f"Loaded {len(eval_samples)} eval samples")

Loaded 19373 train samples
Loaded 195 eval samples


# Train the model

In [23]:
# Init the model
model = Vits.init_from_config(config)

In [24]:
# Init the trainer and 🚀
trainer = Trainer(
    TrainerArgs(restore_path=RESTORE_PATH, skip_train_epoch=SKIP_TRAIN_EPOCH),
    config,
    output_path=OUT_PATH,
    model=model,
    train_samples=train_samples,
    eval_samples=eval_samples,
)
trainer.fit()

fatal: not a git repository (or any parent up to mount point /kaggle)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
fatal: not a git repository (or any parent up to mount point /kaggle)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
 > Training Environment:
 | > Backend: Torch
 | > Mixed precision: True
 | > Precision: fp16
 | > Current device: 0
 | > Num. of GPUs: 1
 | > Num. of CPUs: 4
 | > Num. of Torch Threads: 24
 | > Torch seed: 54321
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: False
 | > Torch TF32 MatMul: False
 > Start Tensorboard: tensorboard --logdir=/kaggle/working/yourtts_tamazight/YourTTS-Tamazight-May-17-2025_08+43PM-0000000
 > Restoring from best_model.pth ...
 > Restoring Model...
 > Partial model initialization...
 | > Layer missing in the model definition: posterior_encoder.enc.in_layers.0.weight_g
 | > Layer missing in the model definition: posterior_encoder.enc.in_

In [25]:
import glob, os
paths = sorted([f for f in glob.glob(OUT_PATH+"/YourTTS*")])
#ckpts = sorted([f for f in glob.glob(OUT_PATH+"/YourTTS*/best_model.pth")])
#configs = sorted([f for f in glob.glob(OUT_PATH+"/YourTTS*/config.json")])
paths

['/kaggle/working/yourtts_tamazight/YourTTS-Tamazight-May-17-2025_08+43PM-0000000']

In [26]:
path = paths[-1]
test_ckpt = os.path.join(path, "best_model.pth")
test_config = os.path.join(path, "config.json")

In [27]:
!tts --text "ⴰⵣⵓⵍ ⴰⵢⵜⵎⴰ ⴷ ⵉⵙⵙⵜⵎⴰ!" \
      --model_path $test_ckpt \
      --config_path $test_config \
      --speaker_idx yan \
      --out_path out.wav

2025-05-18 06:41:43.395204: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747550503.564008    7068 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747550503.617182    7068 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Using model: vits
Setting up Audio Processor...
 | sample_rate: 22050
 | resample: False
 | num_mels: 80
 | log_func: np.log10
 | min_level_db: 0
 | frame_shift_ms: None
 | frame_length_ms: None
 | ref_level_db: None
 | fft_size: 1024
 | power: None
 | preemphasis: 0.0
 | griffin_lim_iters: None
 | signal_norm: None
 | symmetric_norm: None
 | mel_fmin: 0
 | mel_fmax: None
 | pitch_fmin: None
 | pitch_fmax: None
 | spec_gain: 20.0
 | 

In [28]:
import IPython
IPython.display.Audio("out.wav")

In [29]:
# Load the extension and start TensorBoard
%load_ext tensorboard
#%reload_ext tensorboard
%tensorboard --logdir $OUT_PATH

<IPython.core.display.Javascript object>

!curl -sSL https://ngrok-agent.s3.amazonaws.com/ngrok.asc \
  | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null \
  && echo "deb https://ngrok-agent.s3.amazonaws.com buster main" \
  | sudo tee /etc/apt/sources.list.d/ngrok.list \
  && sudo apt update \
  && sudo apt install ngrok

In [30]:
#!ngrok config add-authtoken 2wxfy0o2SaHiJQpWcTKawpLZ5jJ_4az7tDkNynTsk5LXe3nqv

In [31]:
#!ngrok http 6006